In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import torchvision
import torchvision.datasets as datasets
from torch import nn
from torch import optim

device='cpu'

In [ ]:
mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=None)
mnist_testset = datasets.MNIST(root='./data', train=False, download=True, transform=None)

In [ ]:
X_te=mnist_testset.data               ## Test set images
y_te=mnist_testset.test_labels        ## Test set labels

N_o=10                                ## Number of output nodes/classes
N_te=y_te.size()[0]                   ## Number of test samples
Y_te=torch.zeros([N_te,N_o])          ## Initialisation of the one-hot encoded labels for the test set
Y_te[np.arange(0,N_te),y_te]=1        ## From labels to one-hot encoded labels for the test set

X_tr=mnist_trainset.data              ## Train set images
y_tr=mnist_trainset.train_labels      ## Train labels 
N_tr=y_tr.size()[0]                   

Y_tr=torch.zeros([N_tr,N_o])          ## Initialisation of one-hot encoded labels for training
Y_tr[np.arange(0,N_tr),y_tr]=1        ## From labels to one-hot encoded labels for the training set

N_val=10000                           ## Here I take out N_val samples from the training set and use them for validation
i_val=np.random.permutation(np.arange(0,N_tr))[0:N_val]

X_val=X_tr[i_val,:,:]
Y_val=Y_tr[i_val,:]

i_tr=np.delete(np.arange(0,N_tr),i_val)
N_tr=N_tr-N_val

X_tr=X_tr[i_tr,:,:]
Y_tr=Y_tr[i_tr,:]

T=X_tr.size()[2]
N_in=X_tr.size()[1]

## Normalisation and conversion to float
X_M=255

X_tr=(X_tr.float()/X_M).to(device)
X_val=(X_val.float()/X_M).to(device)
X_te=(X_te.float()/X_M).to(device)

Y_tr=torch.tile(Y_tr.float().unsqueeze(2),[1,1,X_tr.size()[2]]).to(device)
Y_val=torch.tile(Y_val.float().unsqueeze(2),[1,1,X_tr.size()[2]]).to(device)
Y_te=torch.tile(Y_te.float().unsqueeze(2),[1,1,X_tr.size()[2]]).to(device)


In [ ]:
class Model_James(nn.Module):
    
    def __init__(self,N,N_in,N_av,alpha,rho,gamma,Ns,fb_ind):
        super().__init__()
        
        
        self.N=N
        self.alpha=torch.tensor(alpha,device=device)
        self.rho=rho
        self.N_av=N_av
        self.N_in=N_in
        self.gamma=gamma
        
        diluition=1-N_av/N
        W=np.random.uniform(-1,1,[N,N])
        W=W*(np.random.uniform(0,1,[N,N])>diluition)
        eig=np.linalg.eigvals(W)
        self.W=torch.from_numpy(self.rho*W/(np.max(np.absolute(eig)))).float().to(device)
        
        self.x=[]
        
        if self.N_in==1:
            
            self.W_in=2*np.random.randint(0,2,[self.N_in,self.N])-1
            self.W_in=torch.from_numpy(self.W_in*self.gamma,device=device).float()
            
            
        else:
            
            self.W_in=np.random.randn(self.N_in,self.N)
            self.W_in=torch.from_numpy(self.gamma*self.W_in).float().to(device)
            
        
        self.Ws=[]
        self.bs=[]
        self.Ns=Ns
        
        for n in range(1,np.shape(Ns)[0]):
            
            self.Ws.append(nn.Parameter(torch.randn([Ns[n-1],Ns[n]],device=device)/torch.sqrt(Ns[n-1]+Ns[n])))
            self.bs.append(nn.Parameter(torch.zeros([Ns[n]],device=device)))
        
        self.fb_ind=fb_ind
        
        self.W_fb=[]
        self.W_fb.append(nn.Parameter(torch.randn([Ns[fb_ind],Ns[0]],device=device)/torch.sqrt(Ns[n-1]+Ns[n])))
    
    def Initialize_Hyper(self,eta):
        
        self.eta=eta
        
        self.opt=optim.Adam([{ 'params': self.Ws+self.bs+self.W_fb, 'lr':eta }])
    
    
    def Reset(self,s):
        
        batch_size=s.size()[0]
        self.x=torch.zeros([batch_size,self.N],device=device)
        self.xs=[]
        self.xs.append(self.x)
        
        for n in range(1,np.shape(self.Ns)[0]-1):
            
            self.xs.append(torch.zeros([batch_size,self.Ns[n]],device=device))
            if n==self.fb_ind:
                
                self.fb=torch.clone(self.xs[n])
        
    def Forward(self,s):
        
        
        self.x=(1-self.alpha)*self.x+self.alpha*torch.tanh(torch.matmul(s,self.W_in)+torch.matmul(self.x,self.W)+\
                                                          torch.matmul(self.fb,self.W_fb[0]))
        
        self.xs[0]=self.x
        for n in range(1,np.shape(self.Ns)[0]-1):
            

            self.xs[n]=torch.relu(torch.matmul(self.xs[n-1],self.Ws[n-1])+self.bs[n-1])
            
            if n==self.fb_ind:
                
                self.fb=torch.clone(self.xs[n])
    
        
    def Response(self,Input,Targets):
        
        T=Input.shape[2]
        Out=torch.zeros([Input.shape[0],self.Ns[-1],T],device=device)
        
        self.Reset(Input[:,0,0])
        
        loss=nn.BCEWithLogitsLoss()
        
        for t in range(T):

            self.Forward(Input[:,:,t])
            
            Out[:,:,t]=torch.matmul(self.xs[-1],self.Ws[-1])+self.bs[-1]
            
        Err=loss(Out[:,:,-1],Targets[:,:,-1])

        Err.backward(retain_graph=True)

        Acc=torch.mean( torch.eq(torch.argmax(Targets[:,:,-1],1),torch.argmax(Out[:,:,-1],1)).float() )                           

        self.opt.step()
        self.opt.zero_grad()
            
        return Out, Err, Acc
    
    def Set_Transfer(self,Ns_transfer,eta_transfer):
        
        self.Ws_t=[]
        self.bs_t=[]
        self.Ns_transfer=Ns_transfer
        
        for n in range(1,np.shape(Ns_transfer)[0]):
            
            self.Ws_t.append(nn.Parameter(torch.randn([Ns_transfer[n-1],Ns_transfer[n]],device=device)/torch.sqrt(Ns_transfer[n-1]+Ns_transfer[n])))
            self.bs_t.append(nn.Parameter(torch.zeros([Ns_transfer[n]],device=device)))
        
        self.opt_tr=optim.Adam([{ 'params': self.Ws_t+self.bs_t, 'lr':eta_transfer}])

    def Transfer(self,Input,Targets):
        
        T=Input.shape[2]
        X=torch.zeros([Input.shape[0],self.Ns[-2]],device=device)
        self.Reset(Input[:,0,0])
        
        loss=nn.BCEWithLogitsLoss()
        
        with torch.no_grad():
        
            for t in range(T):

                self.Forward(Input[:,:,t])
                
            X=torch.clone(self.xs[-1])
        
        Xs_transfer=[]
        Xs_transfer.append(X)
        
        for n in range(1,np.shape(self.Ns_transfer)[0]-1):
            
            Xs_transfer.append(torch.relu(torch.matmul(self.Xs_transfer[n-1],self.Ws_t[n-1])+self.bs_t[n-1]))
            
        Out=torch.matmul(Xs_transfer[-1],self.Ws_t[-1])+self.bs_t[-1]
        
        Err=loss(Out,Targets[:,:,-1])
        
        Err.backward(retain_graph=True)

        Acc=torch.mean( torch.eq(torch.argmax(Targets[:,:,-1],1),torch.argmax(Out,1)).float() )                           

        self.opt_tr.step()
        self.opt_tr.zero_grad()
            
        return Out, Err, Acc
                           
   
    

In [11]:
N_in=X_tr.size()[1]
N=500
N_av=10
alpha=0.8
rho=0.99
gamma=0.1
Ns=torch.tensor([500,100,10])
fb_ind=1

NN=Model_James(N, N_in, N_av, alpha, rho, gamma, Ns, fb_ind)
eta=0.001
NN.Initialize_Hyper(eta)
N_train=5000
N_transfer=1000
batch_size=50


Err=torch.zeros([N_train])
Acc=torch.zeros([N_train])

for n in range(N_train):
    
    rand_ind=np.random.randint(0,X_tr.size()[0],[batch_size])
    X_b=X_tr[rand_ind,:,:]
    Y_b=Y_tr[rand_ind,:,:]
    
    out, err, acc=NN.Response(X_b,Y_b)
    
    Err[n]=err.detach().to('cpu')
    Acc[n]=acc.detach().to('cpu')
    
    if n%250==0:
        
        print(torch.mean(Err[n-100:n]), torch.mean(Acc[n-100:n]))
        


    

tensor(nan) tensor(nan)
tensor(0.2679) tensor(0.3578)
tensor(0.1595) tensor(0.6690)
tensor(0.1243) tensor(0.7478)
tensor(0.0985) tensor(0.8300)
tensor(0.0766) tensor(0.8698)
tensor(0.0683) tensor(0.8906)
tensor(0.0559) tensor(0.9126)
tensor(0.0547) tensor(0.9072)
tensor(0.0516) tensor(0.9206)
tensor(0.0501) tensor(0.9204)
tensor(0.0474) tensor(0.9236)
tensor(0.0426) tensor(0.9348)
tensor(0.0485) tensor(0.9212)
tensor(0.0387) tensor(0.9426)
tensor(0.0377) tensor(0.9364)
tensor(0.0374) tensor(0.9448)
tensor(0.0322) tensor(0.9488)
tensor(0.0331) tensor(0.9514)
tensor(0.0368) tensor(0.9428)


In [12]:
Ns_transfer=torch.tensor(Ns[-2:])

N_transfer=5000
Err_t=torch.zeros([N_transfer])
Acc_t=torch.zeros([N_transfer])

eta_transfer=0.001
NN.Set_Transfer(Ns_transfer,eta_transfer)


for n in range(N_train):
    
    rand_ind=np.random.randint(0,X_tr.size()[0],[batch_size])
    X_b=X_tr[rand_ind,:,:]
    Y_b=Y_tr[rand_ind,:,:]
    
    out, err, acc=NN.Transfer(X_b,Y_b)
    
    Err_t[n]=err.detach().to('cpu')
    Acc_t[n]=acc.detach().to('cpu')
    
    if n%100==0:
        
        print(torch.mean(Err_t[n-100:n]), torch.mean(Acc_t[n-100:n]))
        

/tmp/ipykernel_11150/1414183574.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Ns_transfer=torch.tensor(Ns[-2:])


tensor(nan) tensor(nan)
tensor(0.3736) tensor(0.4154)
tensor(0.1171) tensor(0.9520)
tensor(0.0708) tensor(0.9544)
tensor(0.0539) tensor(0.9518)
tensor(0.0442) tensor(0.9566)
tensor(0.0376) tensor(0.9598)
tensor(0.0343) tensor(0.9602)
tensor(0.0355) tensor(0.9530)
tensor(0.0328) tensor(0.9552)
tensor(0.0317) tensor(0.9558)
tensor(0.0301) tensor(0.9570)
tensor(0.0265) tensor(0.9624)
tensor(0.0280) tensor(0.9590)
tensor(0.0292) tensor(0.9558)
tensor(0.0252) tensor(0.9634)
tensor(0.0263) tensor(0.9590)
tensor(0.0233) tensor(0.9644)
tensor(0.0247) tensor(0.9614)
tensor(0.0261) tensor(0.9624)
tensor(0.0254) tensor(0.9602)
tensor(0.0232) tensor(0.9626)
tensor(0.0277) tensor(0.9584)
tensor(0.0258) tensor(0.9620)
tensor(0.0236) tensor(0.9630)
tensor(0.0272) tensor(0.9568)
tensor(0.0236) tensor(0.9624)
tensor(0.0258) tensor(0.9598)
tensor(0.0258) tensor(0.9610)
tensor(0.0234) tensor(0.9650)
tensor(0.0236) tensor(0.9638)
tensor(0.0228) tensor(0.9650)
tensor(0.0274) tensor(0.9576)
tensor(0.0255) t